#PREPROCESSING - PART 2

For this second part of the preprocessing, now that the data is unified and has a certain structure, the text will be cleaned and emojis & hashtags will be dealt with. The output of this notebook is, for each language, two csv files: one for data containing emojis and hashtags, and another with these elements removed.

*Note: When saving or loading data from Drive, the paths are specific to my personal Drive*

In [ ]:
import pandas as pd
!pip install emoji
import emoji
import re
import numpy
import warnings
#In order to ignore warnings when overwriting row values in panda dataframes
warnings.filterwarnings('ignore')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 175 kB 11.1 MB/s 
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171046 sha256=9ed6b05a218ed1764c5fbb3eb91feae58f88850d6af03c44f89e2f71c77dd6a4
  Stored in directory: /root/.cache/pip/wheels/8a/4e/b6/57b01db010d17ef6ea9b40300af725ef3e210cb1acfb7ac8b6
Successfully built emoji


In [ ]:
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


####Imports & functions

In [ ]:
#Some dataset's emojis don't show as emojis, we need to replace the emoji code with the emoji in the df
df_emoji_code_to_emoji = pd.read_csv('/content/emoji_codes_csv.csv', encoding='utf8', engine='python')
df_emoji_code_to_emoji = df_emoji_code_to_emoji[["hex code", "emoji_code", "emoji"]]

In [ ]:
#defining functions
def unique(list1):
    x = numpy.array(list1)
    return numpy.unique(x)

#If a hashtag is followed by another hashtag or a special character (?!), this divides it
def divideHashtagtext(txt):
  specialChars = "#”.,@?¿¡!|:" 
  for specialChar in specialChars:
    txt = txt.replace(specialChar, "$$"+specialChar)
  txt = txt.replace(',', ' ')
  return txt.split("$$")

#Deletes accents in a string
def deleteAccents(txt):
    replacements = (
        ("á", "a"),
        ("é", "e"),
        ("í", "i"),
        ("ó", "o"),
        ("ú", "u"),
    )
    for a, b in replacements:
        txt = txt.replace(a, b).replace(a.upper(), b.upper())
    return txt

#Returns all emojis in a string
def check_emojis(tweet : str):
  words = list(tweet)
  listEmojis = str()
  for i in words:
    if emoji.is_emoji(i):
      listEmojis = listEmojis + " " + i
  return listEmojis

#Returns all codes with the structure &#____; (indicating emojis/special symbols)
def getAllEmojiHexCode(df):
  emoji_codes = []
  pattern = "&#(.*?)\;"
  for idx, row in df.iterrows():
    match = re.findall(pattern, row.text)
    for m in match:
      emoji_codes.append( "&#" + m + ";")

  return unique(emoji_codes)

#Replaces hex codes with the corresponding emoji/symbol in a dataframe
def replaceEmojiCodes(df):
  for idx, row in df.iterrows():
    for eidx, erow in df_emoji_code_to_emoji.iterrows():
      row.text = row.text.replace(df_emoji_code_to_emoji['hex code'][eidx], str(df_emoji_code_to_emoji['emoji'][eidx])) 
      df['text'][idx] = row.text

  return df

#Returns keys of a dictionary whose value is superior to "num"
def important_values(dict_, num):
  dict_important = {}
  for (key, value) in dict_.items():
    if value > num:
        dict_important[key] = value
  return dict_important

#Replace all emojis in keys_rep in dataframe for text in between colons, and delete al emojis in keys_del in same dataframe
def replace_emojis(keys_rep, keys_del, df):
  for idx, row in df.iterrows():
    for k in keys_del:
      row.text = row.text.replace(k, "")
      df['text'][idx] = row.text
    for k in keys_rep:
      row.text = row.text.replace(k, " " + emoji.demojize(k) + " ")
      df['text'][idx] = row.text
  return df

In [ ]:
#This function returns a cleaned dataframe, plus dictionaries with all the hashtags and emojis in them
def cleanTweets_getDicts(df):
  #Just choose spanish tweets
  hashtag_dict = {}
  emoji_dict = {}
  #Change urls and mentions for a single word
  for idx, row in df.iterrows():
        #Remove ""
        row.text = row.text.strip('"')
        df['text'][idx] = row.text
        #
        all_emojis = check_emojis(row.text)
        all_ats = list(filter(lambda word: word[0]=='@', row.text.split())) 
        all_url = list(filter(lambda word: word[:8]=='https://', row.text.split())) + list(filter(lambda word: word[:7]=='http://', row.text.split()))
        all_hashtags = list(filter(lambda word: word[0]=='#', row.text.split()))   
        ##Deletes indication that the tweet is a Retweet
        if row.text.startswith('RT '):
          row.text = row.text[3:];
          df['text'][idx] = row.text
        ##
        for i in range(len(all_hashtags)):
            all_hashtags[i] = all_hashtags[i]
            if len(divideHashtagtext(all_hashtags[i])) <= 2:
              if deleteAccents(all_hashtags[i]) in hashtag_dict:
                hashtag_dict[deleteAccents(all_hashtags[i])] = hashtag_dict[deleteAccents(all_hashtags[i])] + 1
              else:
                hashtag_dict[deleteAccents(all_hashtags[i])] = 1
            else:
              hashtag_aux_list = divideHashtagtext(all_hashtags[i])
              hashtag_aux_list = [i for i in hashtag_aux_list if i.startswith('#')]
              for string in hashtag_aux_list:
                if string[0] == '#':
                  if deleteAccents(string) in hashtag_dict:
                    hashtag_dict[deleteAccents(string)] = hashtag_dict[deleteAccents(string)] + 1
                  else:
                    hashtag_dict[deleteAccents(string)] = 1
                elif string[0] == '@':
                  row.text = row.text.replace(string, "") #MENTION
                  df['text'][idx] = row.text
        for mention in all_ats:
          row.text = row.text.replace(mention, "") #MENTION
          df['text'][idx] = row.text
        for url in all_url:
          row.text = row.text.replace(url, "") #URL
          df['text'][idx] = row.text
        for e in check_emojis(row.text):
            if e in emoji_dict:
              emoji_dict[e] = emoji_dict[e] + 1
            else:
              emoji_dict[e] = 1
              
  return df, hashtag_dict, emoji_dict

#SPANISH DATA

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/TFG/data/final_data/unfinished_spanish_data.csv', encoding='utf8', engine='python')

####Emoji code extraction and replacement

In [ ]:
emoji_codes = getAllEmojiHexCode(df)

In [ ]:
emoji_codes

array([], dtype=float64)

In [ ]:
df = replaceEmojiCodes(df)

####Hashtag and emoji filtering

In [ ]:
df, hashtag_dict, emoji_dict = cleanTweets_getDicts(df)

In [ ]:
emo_keys = list(emoji_dict.keys())
emo_keys.remove(" ")

In [ ]:
hash_keys = list(hashtag_dict.keys())
hash_keys.remove("#")

#####Removal of all emojis and hashtags

In [ ]:
#Remove emojis
df = replace_emojis([], emo_keys, df)

In [ ]:
#Make sure no hashtag is left in data by mistake
for idx, row in df.iterrows():
      #We do this loop now and not in cells above because we leave a space before every #, and it could cause problems with emojis/special characters
      row.text = row.text.replace("#", " #") 
      df['text'][idx] = row.text
      all_hashtags = list(filter(lambda word: word[0]=='#', row.text.split()))   
      for i in range(len(all_hashtags)):
          hash_keys = hash_keys + all_hashtags
hash_keys = list(dict.fromkeys(hash_keys))
for c in [",","!",":",".","¿","|","?","¡","'"]:
  hash_keys = [s.strip(c) for s in hash_keys]
hash_keys = sorted(hash_keys, key=len, reverse=True)

In [ ]:
#Remove hashtags 
for k in hash_keys:
  df['text'] = df.apply(lambda row: row.text.replace(k, ""), axis=1)

In [ ]:
df.to_csv('/content/drive/MyDrive/TFG/data/final_data/removed_spanish_data.csv', index=False)  

#####All emojis and hashtags kept

In [ ]:
#Replace emojis with text
df = replace_emojis(emo_keys, [], df)

In [ ]:
df.to_csv('/content/drive/MyDrive/TFG/data/final_data/mantained_spanish_data.csv', index=False)  

#ITALIAN DATA

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/TFG/data/final_data/unfinished_italian_data.csv', encoding='utf8', engine='python')

####Emoji code extraction and replacement

In [ ]:
emoji_codes = getAllEmojiHexCode(df)

In [ ]:
emoji_codes

In [ ]:
df = replaceEmojiCodes(df)

In [ ]:
df.to_csv('/content/drive/MyDrive/TFG/data/final_data/unfinished_replacedemojis_italian_data.csv', index=False) 


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/TFG/data/final_data/unfinished_replacedemojis_italian_data.csv', encoding='utf8', engine='python')

####Hashtag and emoji filtering

In [ ]:
df, hashtag_dict, emoji_dict = cleanTweets_getDicts(df)

In [ ]:
emo_keys = list(emoji_dict.keys())
emo_keys.remove(" ")

In [ ]:
hash_keys = list(hashtag_dict.keys())
hash_keys.remove("#")
hash_keys = sorted(hash_keys, key=len, reverse=True)

#####Removal of all emojis and hashtags

In [ ]:
#Remove emojis
df = replace_emojis([], emo_keys, df)

In [ ]:
df

,text,hate speech
0,Fatti trovare te lo do volentieri e ti sbor...,1.0
1,Tu dovresti ricominciare dai semafori a fare ...,1.0
2,"Amore,sei presentabile? Xchè così via Skype ti...",1.0
3,"Salvo poi mandare la culona a Mosca, aummaumm...",1.0
4,Ti sborro io,1.0
...,...,...
17446,Gli stati nazionali devono essere pronti a rin...,0.0
17447,Il ministro dell'interno della Germania #Horst...,0.0
17448,#Salvini: In Italia troppi si sono montati la ...,0.0
17449,Chi giubila in buona fede non ha capito nien...,0.0


In [ ]:
#Make sure no hashtag is left in data by mistake
for idx, row in df.iterrows():
      #We do this loop now and not in cells above because we leave a space before every #, and it could cause problems with emojis/special characters
      row.text = row.text.replace("#", " #") 
      df['text'][idx] = row.text
      all_hashtags = list(filter(lambda word: word[0]=='#', row.text.split()))   
      for i in range(len(all_hashtags)):
          hash_keys = hash_keys + all_hashtags
hash_keys = list(dict.fromkeys(hash_keys))
for c in [",","!",":",".","¿","|","?","¡","'"]:
  hash_keys = [s.strip(c) for s in hash_keys]
hash_keys = sorted(hash_keys, key=len, reverse=True)

In [ ]:
#Remove hashtags 
for k in hash_keys:
  df['text'] = df.apply(lambda row: row.text.replace(k, ""), axis=1)

In [ ]:
df.to_csv('/content/drive/MyDrive/TFG/data/final_data/removed_italian_data.csv', index=False)  

#####All emojis and hashtags kept

In [ ]:
#Replace emojis with text
df = replace_emojis(emo_keys, [], df)

In [ ]:
df.to_csv('/content/drive/MyDrive/TFG/data/final_data/mantained_italian_data.csv', index=False)  

#PORTUGUESE DATA

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/TFG/data/final_data/unfinished_portuguese_data.csv', encoding='utf8', engine='python')

####Emoji code extraction and replacement

In [ ]:
emoji_codes = getAllEmojiHexCode(df)

In [ ]:
emoji_codes

array([], dtype=float64)

In [ ]:
df = replaceEmojiCodes(df)

####Hashtag and emoji filtering

In [ ]:
df, hashtag_dict, emoji_dict = cleanTweets_getDicts(df)

In [ ]:
emo_keys = list(emoji_dict.keys())
emo_keys.remove(" ")

In [ ]:
hash_keys = list(hashtag_dict.keys())
hash_keys.remove("#")
hash_keys = sorted(hash_keys, key=len, reverse=True)

#####Removal of all emojis and hashtags

In [ ]:
#Remove emojis
df = replace_emojis([], emo_keys, df)

In [ ]:
#Make sure no hashtag is left in data by mistake
for idx, row in df.iterrows():
      #We do this loop now and not in cells above because we leave a space before every #, and it could cause problems with emojis/special characters
      row.text = row.text.replace("#", " #") 
      df['text'][idx] = row.text
      all_hashtags = list(filter(lambda word: word[0]=='#', row.text.split()))   
      for i in range(len(all_hashtags)):
          hash_keys = hash_keys + all_hashtags
hash_keys = list(dict.fromkeys(hash_keys))
for c in [",","!",":",".","¿","|","?","¡","'"]:
  hash_keys = [s.strip(c) for s in hash_keys]
hash_keys = sorted(hash_keys, key=len, reverse=True)

In [ ]:
#Remove hashtags
for k in hash_keys:
  df['text'] = df.apply(lambda row: row.text.replace(k, ""), axis=1)

In [ ]:
df.to_csv('/content/drive/MyDrive/TFG/data/final_data/removed_portuguese_data.csv', index=False)  

#####All emojis and hashtags kept

In [ ]:
#Replace emojis with text
df = replace_emojis(emo_keys, [], df)

In [ ]:
df.to_csv('/content/drive/MyDrive/TFG/data/final_data/mantained_portuguese_data.csv', index=False)  

#ENGLISH DATA

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/TFG/data/final_data/unfinished_english_data.csv', encoding='utf8', engine='python')

####Data splitting
The data is split into 5 equal parts and then re-unified. This is done because runtime would be too long in some cells for the whole data

In [ ]:
df1 = df.sample(frac = 0.2)
df = df.drop(df1.index)
df1.to_csv('/content/drive/MyDrive/TFG/data/final_data/unfinished_english_data_1.csv', index=False) 
#
df2 = df.sample(frac = 0.25)
df = df.drop(df2.index)
df2.to_csv('/content/drive/MyDrive/TFG/data/final_data/unfinished_english_data_2.csv', index=False) 
#
df3 = df.sample(frac = 0.3)
df = df.drop(df3.index)
df3.to_csv('/content/drive/MyDrive/TFG/data/final_data/unfinished_english_data_3.csv', index=False) 
#
df4 = df.sample(frac = 0.5)
df5 = df.drop(df4.index)
df4.to_csv('/content/drive/MyDrive/TFG/data/final_data/unfinished_english_data_4.csv', index=False) 
df5.to_csv('/content/drive/MyDrive/TFG/data/final_data/unfinished_english_data_5.csv', index=False) 

In [ ]:
#@title Choose subsection to run the preprocessing code on

Subdataframe = '5'  #@param ["1", "2", "3", "4", "5"]

map_subsection_load_path = {
    '1':
        '/content/drive/MyDrive/TFG/data/final_data/unfinished_english_data_1.csv',
    '2':
        '/content/drive/MyDrive/TFG/data/final_data/unfinished_english_data_2.csv',
    '3':
        '/content/drive/MyDrive/TFG/data/final_data/unfinished_english_data_3.csv',
    '4':
        '/content/drive/MyDrive/TFG/data/final_data/unfinished_english_data_4.csv',
    '5':
        '/content/drive/MyDrive/TFG/data/final_data/unfinished_english_data_5.csv',
}

map_replaced_emoji_save_path = {
    '1':
        '/content/drive/MyDrive/TFG/data/final_data/unfinished_replacedemojis_english_data_1.csv',
    '2':
        '/content/drive/MyDrive/TFG/data/final_data/unfinished_replacedemojis_english_data_2.csv',
    '3':
        '/content/drive/MyDrive/TFG/data/final_data/unfinished_replacedemojis_english_data_3.csv',
    '4':
        '/content/drive/MyDrive/TFG/data/final_data/unfinished_replacedemojis_english_data_4.csv',
    '5':
        '/content/drive/MyDrive/TFG/data/final_data/unfinished_replacedemojis_english_data_5.csv',
}

map_removed_save_path = {
    '1':
        '/content/drive/MyDrive/TFG/data/final_data/removed_english_data_1.csv',
    '2':
        '/content/drive/MyDrive/TFG/data/final_data/removed_english_data_2.csv',
    '3':
        '/content/drive/MyDrive/TFG/data/final_data/removed_english_data_3.csv',
    '4':
        '/content/drive/MyDrive/TFG/data/final_data/removed_english_data_4.csv',
    '5':
        '/content/drive/MyDrive/TFG/data/final_data/removed_english_data_5.csv',
}

map_filtered_save_path = {
    '1':
        '/content/drive/MyDrive/TFG/data/final_data/filtered_english_data_1.csv',
    '2':
        '/content/drive/MyDrive/TFG/data/final_data/filtered_english_data_2.csv',
    '3':
        '/content/drive/MyDrive/TFG/data/final_data/filtered_english_data_3.csv',
    '4':
        '/content/drive/MyDrive/TFG/data/final_data/filtered_english_data_4.csv',
    '5':
        '/content/drive/MyDrive/TFG/data/final_data/filtered_english_data_5.csv',
}

map_mantained_save_path = {
    '1':
        '/content/drive/MyDrive/TFG/data/final_data/mantained_english_data_1.csv',
    '2':
        '/content/drive/MyDrive/TFG/data/final_data/mantained_english_data_2.csv',
    '3':
        '/content/drive/MyDrive/TFG/data/final_data/mantained_english_data_3.csv',
    '4':
        '/content/drive/MyDrive/TFG/data/final_data/mantained_english_data_4.csv',
    '5':
        '/content/drive/MyDrive/TFG/data/final_data/mantained_english_data_5.csv',
}

load_path = map_subsection_load_path[Subdataframe]
emoji_save_path = map_replaced_emoji_save_path[Subdataframe]
removed_path = map_removed_save_path[Subdataframe]
filtered_path = map_filtered_save_path[Subdataframe]
mantained_path = map_mantained_save_path[Subdataframe]

print(f'Data to load           : {load_path}')

Data to load           : /content/drive/MyDrive/TFG/data/final_data/unfinished_english_data_5.csv


In [ ]:
df = pd.read_csv(load_path, encoding='utf8', engine='python')

####Emoji code extraction and replacement

In [ ]:
emoji_codes = getAllEmojiHexCode(df)

In [ ]:
emoji_codes

array(['&#10067;', '&#10084;', '&#1040784;', '&#1041184;', '&#1041185;',
       '&#1041191;', '&#1041204;', '&#1041219;', '&#1041653;',
       '&#1043215;', '&#1043291;', '&#1043292;', '&#1043323;',
       '&#1043359;', '&#11015;', '&#12288;', '&#12290;', '&#127383;',
       '&#127470;', '&#127480;', '&#127481;', '&#127482;', '&#127752;',
       '&#127754;', '&#127797;', '&#127809;', '&#127811;', '&#127814;',
       '&#127825;', '&#127829;', '&#127831;', '&#127849;', '&#127850;',
       '&#127856;', '&#127864;', '&#127866;', '&#127867;', '&#127874;',
       '&#127876;', '&#127877;', '&#127880;', '&#127881;', '&#127882;',
       '&#127908;', '&#127911;', '&#127926;', '&#127936;', '&#127938;',
       '&#127939;', '&#127944;', '&#127969;', '&#127984;', '&#128014;',
       '&#128025;', '&#128027;', '&#128029;', '&#128034;', '&#128035;',
       '&#128048;', '&#128049;', '&#128056;', '&#128064;', '&#128066;',
       '&#128068;', '&#128069;', '&#128070;', '&#128072;', '&#128073;',
       '&#1

In [ ]:
df = replaceEmojiCodes(df)

In [ ]:
df.to_csv(emoji_save_path, index=False) 


In [ ]:
df = pd.read_csv(emoji_save_path, encoding='utf8', engine='python')

####Hashtag and emoji filtering

In [ ]:
df, hashtag_dict, emoji_dict = cleanTweets_getDicts(df)

In [ ]:
emo_keys = list(emoji_dict.keys())
emo_keys.remove(" ")

In [ ]:
hash_keys = list(hashtag_dict.keys())
hash_keys.remove("#")
hash_keys = sorted(hash_keys, key=len, reverse=True)

#####Removal of all emojis and hashtags

In [ ]:
#Remove emojis
df = replace_emojis([], emo_keys, df)

In [ ]:
#Make sure no hashtag is left in data by mistake
for idx, row in df.iterrows():
      #We do this loop now and not in cells above because we leave a space before every #, and it could cause problems with emojis/special characters
      row.text = row.text.replace("#", " #") 
      df['text'][idx] = row.text
      all_hashtags = list(filter(lambda word: word[0]=='#', row.text.split()))   
      for i in range(len(all_hashtags)):
          hash_keys = hash_keys + all_hashtags
hash_keys = list(dict.fromkeys(hash_keys))
for c in [",","!",":",".","¿","|","?","¡","'"]:
  hash_keys = [s.strip(c) for s in hash_keys]
hash_keys = sorted(hash_keys, key=len, reverse=True)

In [ ]:
#Remove hashtags 
for k in hash_keys:
  df['text'] = df.apply(lambda row: row.text.replace(k, ""), axis=1)

In [ ]:
df.to_csv(removed_path, index=False)  

#####All emojis and hashtags kept

In [ ]:
#Replace emojis with text
df = replace_emojis(emo_keys, [], df)

In [ ]:
df.to_csv(mantained_path, index=False)  

####Data unification

In [ ]:
df1 = pd.read_csv('/content/drive/MyDrive/TFG/data/final_data/removed_english_data_1.csv', encoding='utf8', engine='python')
df2 = pd.read_csv('/content/drive/MyDrive/TFG/data/final_data/removed_english_data_2.csv', encoding='utf8', engine='python')
df3 = pd.read_csv('/content/drive/MyDrive/TFG/data/final_data/removed_english_data_3.csv', encoding='utf8', engine='python')
df4 = pd.read_csv('/content/drive/MyDrive/TFG/data/final_data/removed_english_data_4.csv', encoding='utf8', engine='python')
df5 = pd.read_csv('/content/drive/MyDrive/TFG/data/final_data/removed_english_data_5.csv', encoding='utf8', engine='python')

df = pd.concat([df1, df2, df3, df4, df5])

df.to_csv('/content/drive/MyDrive/TFG/data/final_data/removed_english_data.csv', index=False)  

In [ ]:
df1 = pd.read_csv('/content/drive/MyDrive/TFG/data/final_data/filtered_english_data_1.csv', encoding='utf8', engine='python')
df2 = pd.read_csv('/content/drive/MyDrive/TFG/data/final_data/filtered_english_data_2.csv', encoding='utf8', engine='python')
df3 = pd.read_csv('/content/drive/MyDrive/TFG/data/final_data/filtered_english_data_3.csv', encoding='utf8', engine='python')
df4 = pd.read_csv('/content/drive/MyDrive/TFG/data/final_data/filtered_english_data_4.csv', encoding='utf8', engine='python')
df5 = pd.read_csv('/content/drive/MyDrive/TFG/data/final_data/filtered_english_data_5.csv', encoding='utf8', engine='python')

df = pd.concat([df1, df2, df3, df4, df5])

df.to_csv('/content/drive/MyDrive/TFG/data/final_data/filtered_english_data.csv', index=False)  

In [ ]:
df1 = pd.read_csv('/content/drive/MyDrive/TFG/data/final_data/mantained_english_data_1.csv', encoding='utf8', engine='python')
df2 = pd.read_csv('/content/drive/MyDrive/TFG/data/final_data/mantained_english_data_2.csv', encoding='utf8', engine='python')
df3 = pd.read_csv('/content/drive/MyDrive/TFG/data/final_data/mantained_english_data_3.csv', encoding='utf8', engine='python')
df4 = pd.read_csv('/content/drive/MyDrive/TFG/data/final_data/mantained_english_data_4.csv', encoding='utf8', engine='python')
df5 = pd.read_csv('/content/drive/MyDrive/TFG/data/final_data/mantained_english_data_5.csv', encoding='utf8', engine='python')

df = pd.concat([df1, df2, df3, df4, df5])

df.to_csv('/content/drive/MyDrive/TFG/data/final_data/mantained_english_data.csv', index=False)  